# Generalizing Optimal Arbitrage to N-Hop Paths via Möbius Transformations

This notebook generalizes the closed-form arbitrage formula from the [original single-hop analysis](https://medium.com/@jeffreyhartigan/deriving-a-closed-form-solution-for-optimal-dex-arbitrage-b7c917bbe3e7) to arbitrary multi-hop paths across constant product AMMs. The key insight: each AMM swap is a **Möbius transformation**, and Möbius transformations compose via matrix multiplication. This means the entire n-hop path collapses to a single fractional-linear function, preserving the closed-form structure regardless of path length.

The result is a general formula that computes the profit-maximizing trade size for any n-hop circular arbitrage in $O(n)$ time with **zero iteration** — extending the original $O(1)$ two-pool formula to paths of arbitrary length.

In [1]:
import numpy as np
from numpy import sqrt
import time

## Recap: Single-Hop Closed Form

In the [original analysis](optimal_arbitrage.ipynb), we derived the profit-maximizing input for a two-pool arbitrage. Starting from the constant product equations with fee factor $k$:

$$y = b - \frac{ab}{a + kx}, \quad z = c - \frac{cd}{d + ky}$$

We found that setting $\frac{dz}{dx} = 1$ and solving for $x$ yields:

$$x_{\text{ideal}} = \frac{k\sqrt{abcd} - ad}{k(bk + d)}$$

where $(a, b)$ and $(c, d)$ are pool reserves and $k = 0.997$ is the fee factor. This is an exact $O(1)$ result — no iteration, no approximation.

**The question: can this be generalized to n hops?**

The original writeup noted that multi-hop paths produce "higher-degree polynomials under the square root" and "may not admit clean closed-form solutions." It turns out this concern was unfounded. The algebra has hidden structure that keeps the solution clean for any number of hops.

## The Key Insight: AMM Swaps as Möbius Transformations

A single constant product swap with input reserve $r$, output reserve $s$, and fee factor $f$ produces:

$$\text{output} = \frac{s \cdot f \cdot \text{input}}{r + f \cdot \text{input}}$$

This is algebraically equivalent to the formula $y = b - \frac{ab}{a + kx}$ from the original derivation:

$$y = b - \frac{ab}{a + kx} = \frac{b(a + kx) - ab}{a + kx} = \frac{bkx}{a + kx}$$

This function has the form $\frac{\alpha x}{\beta + \gamma x}$, which is a **Möbius transformation** (also called a linear fractional transformation) — specifically one that maps $0 \to 0$.

Möbius transformations have a remarkable algebraic property: **the composition of any two Möbius transformations is again a Möbius transformation.** Furthermore, because every AMM swap sends $0 \to 0$ (zero input always gives zero output), the composed function after any number of hops retains this special form:

$$l(x) = \frac{Kx}{M + Nx}$$

where $K$, $M$, $N$ are constants determined solely by the pool reserves and fees along the path. No matter how many hops — 2, 5, 50 — the output function always has this same simple structure.

## Matrix Representation

Each hop can be encoded as a $2 \times 2$ matrix. The Möbius transformation $f(x) = \frac{sf \cdot x}{r + f \cdot x}$ corresponds to:

$$\mathbf{M}_i = \begin{pmatrix} s_i f_i & 0 \\ f_i & r_i \end{pmatrix}$$

The matrix acts on $x$ via the rule: if $\mathbf{M} = \begin{pmatrix} a & b \\ c & d \end{pmatrix}$, then $f(x) = \frac{ax + b}{cx + d}$.

Composing $n$ hops is simply **matrix multiplication** (applied right-to-left, matching function composition order):

$$\mathbf{M}_{\text{total}} = \mathbf{M}_n \cdot \mathbf{M}_{n-1} \cdots \mathbf{M}_2 \cdot \mathbf{M}_1 = \begin{pmatrix} K & 0 \\ N & M \end{pmatrix}$$

The top-right entry remains exactly zero throughout the multiplication — a consequence of every individual map fixing the origin ($0 \to 0$).

The composed output is then:

$$l(x) = \frac{Kx}{M + Nx}$$

This is the entire multi-hop transfer function encoded in three numbers.

## The 3-Number Recurrence

The matrix product can be computed without storing full matrices. We maintain just three scalars $K$, $M$, $N$ and update them one hop at a time.

**Initialize** with the first hop (input reserve $r_1$, output reserve $s_1$, fee $f_1$):

$$K = s_1 f_1, \quad M = r_1, \quad N = f_1$$

**Update** for each subsequent hop $i$ with reserves $(r_i, s_i)$ and fee $f_i$:

$$K_{\text{new}} = s_i \cdot f_i \cdot K$$

$$M_{\text{new}} = r_i \cdot M$$

$$N_{\text{new}} = r_i \cdot N + f_i \cdot K$$

**Important:** the update for $N$ uses the value of $K$ **before** it is updated in this step. In code, all three values are computed from the old state before any are overwritten.

After processing all $n$ hops, we have the final $K$, $M$, $N$ and the composed transfer function $l(x) = \frac{Kx}{M + Nx}$.

## General Derivative and Optimal Arbitrage

With $l(x) = \frac{Kx}{M + Nx}$, the derivative is straightforward:

$$\frac{dl}{dx} = \frac{K(M + Nx) - Kx \cdot N}{(M + Nx)^2} = \frac{KM}{(M + Nx)^2}$$

The profit function is $\text{profit}(x) = l(x) - x$, so the optimality condition is:

$$\frac{d}{dx}\text{profit}(x) = \frac{dl}{dx} - 1 = 0$$

$$\frac{KM}{(M + Nx)^2} = 1$$

$$(M + Nx)^2 = KM$$

$$M + Nx = \sqrt{KM} \quad \text{(positive root)}$$

$$\boxed{x_{\text{opt}} = \frac{\sqrt{KM} - M}{N}}$$

This formula works for **any number of hops**. Profitable arbitrage exists when $\frac{K}{M} > 1$ (the initial marginal exchange rate exceeds unity).

**Complexity:** $O(n)$ to compute $K, M, N$ via the recurrence (one pass through the pools), then $O(1)$ for the square root formula. The total is $O(n)$ — linear in the number of hops, with no iteration or numerical solving.

In [2]:
def compose_amm_chain(pools):
    """
    Compute K, M, N for a multi-hop AMM path using the 3-number recurrence.

    Parameters:
        pools: list of tuples (r_in, s_out, fee_factor)
               Each tuple represents one hop in the path.

    Returns:
        K, M, N such that the composed output is l(x) = Kx / (M + Nx)
    """
    r, s, f = pools[0]
    K = s * f
    M = r
    N = f

    for r, s, f in pools[1:]:
        K_new = s * f * K
        M_new = r * M
        N_new = r * N + f * K  # uses K before update
        K, M, N = K_new, M_new, N_new

    return K, M, N


def optimal_arbitrage_multihop(pools):
    """
    Compute the profit-maximizing input for an n-hop arbitrage path.

    Parameters:
        pools: list of tuples (r_in, s_out, fee_factor)

    Returns:
        x_opt: optimal input amount (0 if no profitable arbitrage)
        profit: expected profit at x_opt
    """
    K, M, N = compose_amm_chain(pools)

    # No arbitrage if initial marginal rate <= 1
    if K <= M:
        return 0.0, 0.0

    x_opt = (sqrt(K * M) - M) / N

    # Compute actual profit: l(x_opt) - x_opt
    l_x = K * x_opt / (M + N * x_opt)
    profit = l_x - x_opt

    return x_opt, profit


def simulate_multihop(x, pools):
    """Simulate the actual output of sending x through the multi-hop path."""
    amount = x
    for r, s, f in pools:
        amount = (s * f * amount) / (r + f * amount)
    return amount

## Verification: 2-Hop Case Matches the Original Formula

Using the same SHIB/ETH pool data from the original analysis, the multi-hop recurrence should produce identical results to the single-hop formula $x_{\text{ideal}} = \frac{k\sqrt{abcd} - ad}{k(bk + d)}$.

To map the 2-pool setup into the multi-hop framework:
- **Hop 1:** Input token₀ into Pool A. Input reserve = $a$ (token₀), output reserve = $b$ (token₁), fee = $k$.
- **Hop 2:** Input token₁ into Pool B. Input reserve = $d$ (token₁), output reserve = $c$ (token₀), fee = $k$.

The recurrence gives:
- After hop 1: $K = bk$, $M = a$, $N = k$
- After hop 2: $K = bck^2$, $M = ad$, $N = k(bk + d)$

Plugging into $x_{\text{opt}} = \frac{\sqrt{KM} - M}{N}$:

$$x_{\text{opt}} = \frac{\sqrt{bck^2 \cdot ad} - ad}{k(bk + d)} = \frac{k\sqrt{abcd} - ad}{k(bk + d)}$$

This is **exactly** the original formula.

In [3]:
# Original pool data from the single-hop analysis (SHIB/ETH on Uniswap V2)
a = 15800.025178893529930149   # Token0 in Pool A
b = 30348149.556699            # Token1 in Pool A
c = 5251.705779226172996106    # Token0 in Pool B
d = 9986593.845926             # Token1 in Pool B
k = 0.997                     # Fee factor (0.3% fee)

# Original closed-form formula (single-hop specific)
x_original = (k * sqrt(a * b * c * d) - a * d) / (k * (b * k + d))

# Multi-hop recurrence with 2 pools
# Hop 1: token0 -> token1 via Pool A  (r=a, s=b, f=k)
# Hop 2: token1 -> token0 via Pool B  (r=d, s=c, f=k)
pools_2hop = [(a, b, k), (d, c, k)]
x_multihop, profit_multihop = optimal_arbitrage_multihop(pools_2hop)

# Compare
print("2-Hop Verification Against Original Formula")
print("=" * 50)
print(f"Original formula:     x = {x_original:.10f}")
print(f"Multi-hop recurrence: x = {x_multihop:.10f}")
print(f"Difference:               {abs(x_original - x_multihop):.2e}")
print(f"\nProfit (analytic):        {profit_multihop:.10f}")

# Cross-check via simulation
actual_output = simulate_multihop(x_multihop, pools_2hop)
print(f"Simulated output:         {actual_output:.10f}")
print(f"Simulated profit:         {actual_output - x_multihop:.10f}")
print(f"Profit match:             {abs(profit_multihop - (actual_output - x_multihop)) < 1e-10}")

2-Hop Verification Against Original Formula
Original formula:     x = 7.9210884160
Multi-hop recurrence: x = 7.9210884160
Difference:               8.88e-16

Profit (analytic):        0.0159546619
Simulated output:         7.9370430779
Simulated profit:         0.0159546619
Profit match:             True


## Multi-Hop Arbitrage: 3, 4, and 5 Hop Paths

To validate the general formula beyond the 2-hop case, we construct synthetic multi-hop circular arbitrage paths. Each path represents a round-trip trade (e.g., Token A $\to$ B $\to$ C $\to$ A) where price discrepancies across pools create a profitable cycle.

For each path, we verify that:
1. The closed-form $x_{\text{opt}} = \frac{\sqrt{KM} - M}{N}$ produces a valid result
2. The analytic profit matches the simulated profit (feeding $x_{\text{opt}}$ through the actual swap chain)
3. The initial marginal rate $K/M > 1$ (confirming arbitrage exists)

In [4]:
# 3-hop path: Token A -> Token B -> Token C -> Token A
# Reserves chosen to create a small round-trip price edge
pools_3hop = [
    (1000.0, 2050000.0, 0.997),    # Pool 1: A/B  (r_in=A, s_out=B)
    (2000000.0, 1020.0, 0.997),    # Pool 2: B/C  (r_in=B, s_out=C)
    (980.0, 1060.0, 0.997),        # Pool 3: C/A  (r_in=C, s_out=A)
]

# 4-hop path: Token A -> B -> C -> D -> A
pools_4hop = [
    (5000.0, 10200000.0, 0.997),   # Pool 1: A/B
    (10000000.0, 5100.0, 0.997),   # Pool 2: B/C
    (4900.0, 9900000.0, 0.997),    # Pool 3: C/D
    (9500000.0, 5250.0, 0.997),    # Pool 4: D/A
]

# 5-hop path: Token A -> B -> C -> D -> E -> A
pools_5hop = [
    (10000.0, 20500000.0, 0.997),  # Pool 1: A/B
    (20000000.0, 10200.0, 0.997),  # Pool 2: B/C
    (9800.0, 20000000.0, 0.997),   # Pool 3: C/D
    (19500000.0, 10100.0, 0.997),  # Pool 4: D/E
    (9700.0, 10500.0, 0.997),      # Pool 5: E/A
]

# Compute and verify for each path length
for name, pools in [("3-hop", pools_3hop), ("4-hop", pools_4hop), ("5-hop", pools_5hop)]:
    K, M, N = compose_amm_chain(pools)
    marginal_rate = K / M
    x_opt, profit = optimal_arbitrage_multihop(pools)

    # Verify via simulation
    actual_output = simulate_multihop(x_opt, pools)
    actual_profit = actual_output - x_opt

    print(f"\n{name} path:")
    print(f"  K/M (initial marginal rate): {marginal_rate:.6f}")
    print(f"  x_opt:                       {x_opt:.6f}")
    print(f"  Analytic profit:             {profit:.6f}")
    print(f"  Simulated profit:            {actual_profit:.6f}")
    print(f"  Match:                       {abs(profit - actual_profit) < 1e-10}")


3-hop path:
  K/M (initial marginal rate): 1.120700
  x_opt:                       19.078671
  Analytic profit:             1.118604
  Simulated profit:            1.118604
  Match:                       True

4-hop path:
  K/M (initial marginal rate): 1.147772
  x_opt:                       85.827103
  Analytic profit:             6.123027
  Simulated profit:            6.123027
  Match:                       True

5-hop path:
  K/M (initial marginal rate): 1.178442
  x_opt:                       162.151803
  Analytic profit:             13.873792
  Simulated profit:            13.873792
  Match:                       True


## Comparison: Closed-Form vs Binary Search (Multi-Hop)

We now generalize the binary search from the original analysis to work with arbitrary hop counts. The binary search uses a numerical derivative of the multi-hop profit function, bisecting until convergence. We benchmark it against the Möbius closed-form solution across all path lengths.

In [5]:
def multihop_profit(x, pools):
    """Calculate profit for input x through a multi-hop path."""
    return simulate_multihop(x, pools) - x

def binary_search_multihop(pools, precision=1e-12, max_iter=1000):
    """Find optimal input via binary search on the multi-hop profit derivative."""
    # Upper bound: minimum input reserve (conservative)
    upper = min(r for r, s, f in pools)
    left, right = 1e-12, upper
    iterations = 0

    for _ in range(max_iter):
        mid = (left + right) / 2
        eps = max(abs(mid) * 1e-8, 1e-12)
        dp = (multihop_profit(mid + eps, pools) - multihop_profit(mid - eps, pools)) / (2 * eps)

        if (right - left) < precision:
            break
        if dp > 0:
            left = mid
        else:
            right = mid
        iterations += 1

    return mid, iterations

In [6]:
# Benchmark all path lengths
all_pools = [
    ("2-hop", pools_2hop),
    ("3-hop", pools_3hop),
    ("4-hop", pools_4hop),
    ("5-hop", pools_5hop),
]

print(f"{'Path':<8} {'Method':<15} {'x_opt':>14} {'Profit':>14} {'Time (us)':>12} {'Iterations':>12}")
print("-" * 80)

for name, pools in all_pools:
    # Binary search (average over 100 runs)
    start = time.time()
    for _ in range(100):
        x_bs, iters = binary_search_multihop(pools)
    t_bs = (time.time() - start) / 100
    profit_bs = multihop_profit(x_bs, pools)

    # Closed-form (average over 100 runs)
    start = time.time()
    for _ in range(100):
        x_cf, profit_cf = optimal_arbitrage_multihop(pools)
    t_cf = (time.time() - start) / 100

    print(f"{name:<8} {'Binary Search':<15} {x_bs:>14.6f} {profit_bs:>14.6f} {t_bs*1e6:>12.1f} {iters:>12}")
    print(f"{name:<8} {'Closed-Form':<15} {x_cf:>14.6f} {profit_cf:>14.6f} {t_cf*1e6:>12.1f} {'0':>12}")
    print()

Path     Method                   x_opt         Profit    Time (us)   Iterations
--------------------------------------------------------------------------------
2-hop    Binary Search         7.921101       0.015955         41.9           54
2-hop    Closed-Form           7.921088       0.015955          0.9            0

3-hop    Binary Search        19.078670       1.118604         47.2           50
3-hop    Closed-Form          19.078671       1.118604          1.0            0

4-hop    Binary Search        85.827103       6.123027         55.9           53
4-hop    Closed-Form          85.827103       6.123027          1.1            0

5-hop    Binary Search       162.151802      13.873792         64.4           54
5-hop    Closed-Form         162.151803      13.873792          1.3            0



Both methods converge to the same optimal input and profit for every path length. The closed-form solution requires zero iterations regardless of the number of hops.

The time advantage of the closed-form grows with path length: binary search calls the full multi-hop simulation at each iteration (cost per iteration scales linearly with $n$), while the closed-form computes $K$, $M$, $N$ in a single $O(n)$ pass and then evaluates one square root.

## Scaling: Performance vs Number of Hops

The closed-form solution is $O(n)$ for computing $K, M, N$ and $O(1)$ for the final formula. Binary search is $O(n \cdot I)$ where $I$ is the number of iterations (typically 40-60). Below we measure how both methods scale from 2 to 20 hops using randomly generated paths.

In [7]:
def generate_random_path(n_hops, k=0.997, seed=42):
    """Generate a random n-hop circular path with guaranteed arbitrage."""
    rng = np.random.RandomState(seed + n_hops)
    pools = []
    for i in range(n_hops):
        r_in = rng.uniform(1000, 50000)
        s_out = rng.uniform(1000000, 50000000)
        pools.append((r_in, s_out, k))

    # Ensure arbitrage exists (K/M > 1) by boosting last pool if needed
    K, M, N = compose_amm_chain(pools)
    if K <= M:
        r, s, f = pools[-1]
        pools[-1] = (r, s * (M / K) * 1.05, f)  # 5% edge

    return pools

# Measure scaling from 2 to 20 hops
hop_counts = list(range(2, 21))
cf_times = []
bs_times = []

for n in hop_counts:
    pools = generate_random_path(n)

    # Closed-form timing (average over 1000 runs)
    start = time.time()
    for _ in range(1000):
        optimal_arbitrage_multihop(pools)
    cf_times.append((time.time() - start) / 1000 * 1e6)

    # Binary search timing (average over 100 runs)
    start = time.time()
    for _ in range(100):
        binary_search_multihop(pools)
    bs_times.append((time.time() - start) / 100 * 1e6)

print(f"{'Hops':>6} {'Closed-Form (us)':>18} {'Binary Search (us)':>20} {'Speedup':>10}")
print("-" * 58)
for n, cf, bs in zip(hop_counts, cf_times, bs_times):
    print(f"{n:>6} {cf:>18.1f} {bs:>20.1f} {bs/cf:>10.1f}x")

  Hops   Closed-Form (us)   Binary Search (us)    Speedup
----------------------------------------------------------
     2                0.9                765.1      858.5x
     3                1.0                 50.8       52.6x
     4                1.1                 56.1       52.5x
     5                1.1                 62.5       56.1x
     6                1.2                 68.8       57.5x
     7                1.3                 78.0       60.3x
     8                1.4                 83.0       59.1x
     9                1.5                 89.0       58.7x
    10                1.6                 96.4       61.5x
    11                1.7                105.0       62.3x
    12                1.8                109.4       62.0x
    13                1.9                116.4       62.6x
    14                1.9                124.9       64.2x
    15                2.0                129.1       65.1x
    16                2.1                151.2       71.6

## GPU Batch Analysis: Evaluating Multiple Arbitrage Paths Simultaneously

In production MEV systems, the bottleneck is not evaluating a single path — it is scanning **thousands of candidate paths simultaneously** across all available pools. Because our Möbius recurrence operates on simple scalar arithmetic, the entire computation **vectorizes trivially** across a batch of independent paths.

Each path is independent — the recurrence for path $i$ reads only from path $i$'s reserves. This is embarrassingly parallel, making it ideal for GPU acceleration.

The GPU parallelism works by promoting the three scalars $K$, $M$, $N$ from scalars to **tensors of shape `(batch_size,)`** — one entry per candidate route. The recurrence loop still iterates sequentially over hops (typically 3–5), but each iteration performs a single element-wise operation that updates all routes simultaneously. There is no inner loop over paths; PyTorch broadcasts the arithmetic across the entire batch in one vectorized call per hop. This is the mechanism of parallelism: not literal $2\times2$ matrix multiplications, but vectorized scalar recurrence over the batch dimension.

Using PyTorch, we evaluate 100,000+ candidate arbitrage routes on GPU. On Apple Silicon Macs, the MPS (Metal Performance Shaders) backend provides native GPU acceleration. GPU wins at ≥100K routes — below that threshold, MPS dispatch and sync overhead swamps the compute, and CPU is actually faster.

In [8]:
try:
    import torch
    TORCH_AVAILABLE = True
    MPS_AVAILABLE = hasattr(torch.backends, 'mps') and torch.backends.mps.is_available()
    CUDA_AVAILABLE = torch.cuda.is_available()

    if MPS_AVAILABLE:
        GPU_DEVICE = torch.device('mps')
        GPU_NAME = "MPS (Apple GPU)"
    elif CUDA_AVAILABLE:
        GPU_DEVICE = torch.device('cuda')
        GPU_NAME = f"CUDA ({torch.cuda.get_device_name(0)})"
    else:
        GPU_DEVICE = None
        GPU_NAME = "No GPU available (CPU-only benchmarks below)"

    print(f"PyTorch {torch.__version__}")
    print(f"GPU: {GPU_NAME}")
except ImportError:
    TORCH_AVAILABLE = False
    MPS_AVAILABLE = False
    CUDA_AVAILABLE = False
    GPU_DEVICE = None
    print("PyTorch not installed. Install with: pip install torch")
    print("GPU benchmarks will be skipped.")

PyTorch 2.10.0
GPU: MPS (Apple GPU)


In [9]:
if TORCH_AVAILABLE:
    def compose_amm_chain_batched(pools_tensor):
        """
        Compute K, M, N for a batch of multi-hop paths.

        Parameters:
            pools_tensor: shape (batch_size, n_hops, 3)
                          where dim 2 = [r_in, s_out, fee_factor]

        Returns:
            K, M, N each of shape (batch_size,)
        """
        batch, n_hops, _ = pools_tensor.shape

        # First hop
        r = pools_tensor[:, 0, 0]
        s = pools_tensor[:, 0, 1]
        f = pools_tensor[:, 0, 2]
        K = s * f
        M = r.clone()
        N = f.clone()

        # Remaining hops (fully vectorized over batch dimension)
        for i in range(1, n_hops):
            r = pools_tensor[:, i, 0]
            s = pools_tensor[:, i, 1]
            f = pools_tensor[:, i, 2]
            K_new = s * f * K
            M_new = r * M
            N_new = r * N + f * K  # uses old K
            K, M, N = K_new, M_new, N_new

        return K, M, N


    def optimal_arbitrage_batched(pools_tensor):
        """
        Compute optimal input for a batch of multi-hop paths.

        Parameters:
            pools_tensor: shape (batch_size, n_hops, 3)

        Returns:
            x_opt: shape (batch_size,) -- optimal inputs (0 where no arb)
            profit: shape (batch_size,) -- expected profits
        """
        K, M, N = compose_amm_chain_batched(pools_tensor)

        sqrt_km = torch.sqrt(K * M)
        x_opt = (sqrt_km - M) / N

        # Compute profit
        l_x = K * x_opt / (M + N * x_opt)
        profit = l_x - x_opt

        # Zero out cases with no profitable arbitrage
        no_arb = (K <= M) | (x_opt <= 0)
        x_opt = torch.where(no_arb, torch.zeros_like(x_opt), x_opt)
        profit = torch.where(no_arb, torch.zeros_like(profit), profit)

        return x_opt, profit

    print("Batched GPU functions defined successfully")

Batched GPU functions defined successfully


In [10]:
if TORCH_AVAILABLE:
    def generate_batch_data(batch_size, n_hops, k=0.997, device='cpu', dtype=None):
        """Generate a batch of random multi-hop paths as a tensor.

        Defaults to float64 on CPU for numerical stability (float32 overflows at
        5 hops with s_out up to 50M). MPS doesn't support float64, so GPU runs
        use float32; the timing benchmark remains valid even if values hit inf.
        """
        if dtype is None:
            dev = device if isinstance(device, torch.device) else torch.device(device)
            dtype = torch.float32 if dev.type == 'mps' else torch.float64
        r_in = torch.rand(batch_size, n_hops, 1, device=device, dtype=dtype) * 49000 + 1000
        s_out = torch.rand(batch_size, n_hops, 1, device=device, dtype=dtype) * 49000000 + 1000000
        fees = torch.full((batch_size, n_hops, 1), k, device=device, dtype=dtype)
        return torch.cat([r_in, s_out, fees], dim=2)

    # Benchmark parameters
    batch_sizes = [10, 100, 1_000, 10_000, 100_000, 1_000_000]
    n_hops = 5
    n_warmup = 10
    n_runs_map = {10: 200, 100: 200, 1_000: 100, 10_000: 50, 100_000: 20, 1_000_000: 5}

    # Print header
    header = f"{'Batch Size':>12} {'CPU (us)':>12}"
    if GPU_DEVICE is not None:
        header += f" {'GPU (us)':>12} {'Speedup':>10}"
    print(header)
    print("-" * len(header))

    for batch_size in batch_sizes:
        n_runs = n_runs_map[batch_size]

        # ---- CPU benchmark ----
        pools_cpu = generate_batch_data(batch_size, n_hops, device='cpu')

        for _ in range(n_warmup):
            optimal_arbitrage_batched(pools_cpu)

        start = time.time()
        for _ in range(n_runs):
            optimal_arbitrage_batched(pools_cpu)
        cpu_time = (time.time() - start) / n_runs * 1e6

        row = f"{batch_size:>12} {cpu_time:>12.1f}"

        # ---- GPU benchmark ----
        if GPU_DEVICE is not None:
            pools_gpu = generate_batch_data(batch_size, n_hops, device=GPU_DEVICE)

            # Warmup (critical for GPU)
            for _ in range(n_warmup):
                x, p = optimal_arbitrage_batched(pools_gpu)
                if MPS_AVAILABLE:
                    torch.mps.synchronize()
                elif CUDA_AVAILABLE:
                    torch.cuda.synchronize()

            start = time.time()
            for _ in range(n_runs):
                x, p = optimal_arbitrage_batched(pools_gpu)
                if MPS_AVAILABLE:
                    torch.mps.synchronize()
                elif CUDA_AVAILABLE:
                    torch.cuda.synchronize()
            gpu_time = (time.time() - start) / n_runs * 1e6

            row += f" {gpu_time:>12.1f} {cpu_time/gpu_time:>9.1f}x"

        print(row)

  Batch Size     CPU (us)     GPU (us)    Speedup
-------------------------------------------------


          10         60.2        573.1       0.1x
         100         64.1        626.1       0.1x


        1000         93.7        728.8       0.1x
       10000        353.2        825.3       0.4x


      100000       2714.8       2049.9       1.3x


     1000000      58615.4      18498.0       3.2x


In [11]:
if TORCH_AVAILABLE:
    # Show sample results for 10 simultaneous 5-hop arbitrage evaluations
    torch.manual_seed(42)
    pools_sample = generate_batch_data(10, 5, device='cpu')
    x_opts, profits = optimal_arbitrage_batched(pools_sample)

    print(f"Sample: 10 simultaneous {n_hops}-hop arbitrage evaluations")
    print(f"{'Path':>6} {'x_opt':>14} {'Profit':>14} {'Profitable':>12}")
    print("-" * 50)
    for i in range(10):
        arb = "Yes" if profits[i] > 0 else "No"
        print(f"{i+1:>6} {x_opts[i].item():>14.4f} {profits[i].item():>14.4f} {arb:>12}")

    n_profitable = (profits > 0).sum().item()
    print(f"\n{n_profitable}/10 paths have profitable arbitrage opportunities")

Sample: 10 simultaneous 5-hop arbitrage evaluations
  Path          x_opt         Profit   Profitable
--------------------------------------------------
     1         0.0430  25436189.5291          Yes
     2         0.2844  14545446.0972          Yes
     3         2.1618  18328832.4994          Yes
     4         0.1764   8072398.2011          Yes
     5         0.1953  32528241.8987          Yes
     6         0.4498  15676387.3643          Yes
     7         0.4178  44546959.4982          Yes
     8         0.4423   5632207.3716          Yes
     9         1.0556  47007176.3256          Yes
    10         2.1371  47753978.2429          Yes

10/10 paths have profitable arbitrage opportunities


## Conclusion

The Möbius transformation framework provides a complete generalization of the single-hop closed-form arbitrage formula to paths of arbitrary length:

1. **Each hop** is a Möbius transformation representable as a $2 \times 2$ matrix: $\mathbf{M}_i = \begin{pmatrix} s_i f_i & 0 \\ f_i & r_i \end{pmatrix}$

2. **The full path** composes via matrix multiplication or, equivalently, a 3-number recurrence on $K$, $M$, $N$

3. **The output** always has the form $l(x) = \frac{Kx}{M + Nx}$, regardless of hop count

4. **The optimal input** is $x_{\text{opt}} = \frac{\sqrt{KM} - M}{N}$

5. **The computation vectorizes** trivially across thousands of candidate paths for GPU-accelerated batch scanning

This extends the original $O(1)$ single-hop formula to an $O(n)$ multi-hop formula where $n$ is the number of pools in the path. The closed-form matches binary search results exactly while eliminating all iteration, approximation error, and convergence tuning. Because the recurrence is pure arithmetic on three scalars, it maps naturally to GPU hardware — enabling real-time scanning of massive route spaces in production MEV systems.